In [ ]:
cd ../

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd


train_metadata = pd.read_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

In [ ]:
from data_utils import get_col_names


cols = get_col_names("_strain")
cols

In [ ]:
strains = set()

for col in cols:
    strains.update(train_metadata[col].dropna().unique())
    
strains

In [ ]:
from collections import defaultdict
import pandas as pd

unique_strains = defaultdict(set)

# get unique strains for all cols by lab

for lab_id, grp in train_metadata.groupby("lab_id"):

    for col in cols:
        unique_strains[lab_id].update(grp[col].dropna().unique())
        
pd.DataFrame({
    "lab_id": list(unique_strains.keys()),
    "unique_strains": [list(s) for s in unique_strains.values()]
})

In [ ]:
# unique strains for all mouses by labs

strains_by_labs = defaultdict(lambda: defaultdict(set))

for lab_id, grp in train_metadata.groupby("lab_id"):
    
    for col in cols:
        strains_by_labs[lab_id][col].update(grp[col].dropna().unique())
        
for lab_id, strains in strains_by_labs.items():
    print(lab_id)

    for col, strain in strains.items():
            print(f"{col}: {strain}")

    print()

In [ ]:
for idx, row in train_metadata.iterrows():
    print(f"{row[cols[0]]} || {row[cols[1]]} || {row[cols[2]]} || {row[cols[3]]}")

In [ ]:
# count of each unique strain global


counts = (
    train_metadata[cols]
    .stack(dropna=True)            # long format, drops NA
    .value_counts()                # frequency per unique strain
    .sort_values(ascending=False)
)
print(counts)


In [ ]:
# strain counts by lab 